In [2]:
import pandas as pd
import numpy as np
import pathmagic
import initialise as init
import glob
import sys,os
from functools import reduce

In [3]:
# get list of census files to load

#10, 14 removed
# census_files_to_load = ['01','02','03','04','05','06','07','08','09','11','12','13','15','16','17','18','19','20']
census_files_to_load = ['01','02','07']


census_path = init.raw_path + "/census_data/"

census_files_paths = []

area_code_dict = {'lsoa':[], 'msoa':[]}

index_file_counter = 0

for x in census_files_to_load:
    file_to_append = census_path + "census2021-ts0" + x + "/census2021-ts0" + x + "-lsoa.csv"
    append_file = True
    
    if os.path.exists(file_to_append) == False:
        file_to_append = file_to_append.replace('lsoa', 'msoa')
        if os.path.exists(file_to_append) == False:
            print(x, " has no lsoa or msoa group")
            append_file = False  
        else:
            print(x, " has no lsoa group, so msoa group used")
            area_code_dict['msoa'].append(index_file_counter)
            
    else:
        area_code_dict['lsoa'].append(index_file_counter)
    if append_file:
        census_files_paths.append(file_to_append)
        
    index_file_counter +=1
    
census_files_paths

area_code_dict

07  has no lsoa group, so msoa group used


{'lsoa': [0, 1], 'msoa': [2]}

In [4]:
# load census data
census_dfs = []
for x in census_files_paths:
    print('loading', x)
    census_dfs.append(pd.read_csv(x))

loading C:/Users/farqu/Documents/DS_Masters/DSMP/data/x01_raw/census_data/census2021-ts001/census2021-ts001-lsoa.csv
loading C:/Users/farqu/Documents/DS_Masters/DSMP/data/x01_raw/census_data/census2021-ts002/census2021-ts002-lsoa.csv
loading C:/Users/farqu/Documents/DS_Masters/DSMP/data/x01_raw/census_data/census2021-ts007/census2021-ts007-msoa.csv


In [5]:
# create starting file (all unique lsoa and msoa codes) - required to join msoa to lsoa
master_areacode_file_name = "PCD_OA_LSOA_MSOA_LAD_AUG22_UK_LU.csv"
master_areacode_file = pd.read_csv(census_path + master_areacode_file_name, usecols = ['lsoa11cd', 'msoa11cd'],  encoding = "ISO-8859-1")
master_areacode_file = master_areacode_file[~master_areacode_file.duplicated(subset = 'lsoa11cd', keep = 'first')]
master_areacode_file.dropna(axis = 0, inplace = True)

In [6]:
# join onto master file by msoa if msoa or lsoa if lsoa
for i,census_df in enumerate(census_dfs):
    if i in area_code_dict['lsoa']:
        master_areacode_file = master_areacode_file.merge(census_df, left_on = 'lsoa11cd', right_on = 'geography code', how = 'outer')
    
    elif i in area_code_dict['msoa']:
        master_areacode_file = master_areacode_file.merge(census_df, left_on = 'msoa11cd', right_on = 'geography code', how = 'outer')


In [7]:
master_areacode_file.isna().sum()

lsoa11cd                                          4234
msoa11cd                                          4234
date_x                                           11183
geography_x                                      11183
geography code_x                                 11183
                                                 ...  
Age: Aged 96 years; measures: Value              12623
Age: Aged 97 years; measures: Value              12623
Age: Aged 98 years; measures: Value              12623
Age: Aged 99 years; measures: Value              12623
Age: Aged 100 years and over; measures: Value    12623
Length: 145, dtype: int64

In [14]:
master_areacode_file[~master_areacode_file['geography_x'].isna()]['geography_x']


680           St Albans 014B
681           St Albans 014C
682           St Albans 014A
683           St Albans 014D
684           St Albans 013C
                ...         
44641           Newport 019E
44642           Newport 019F
44643    Merthyr Tydfil 005E
44644    Merthyr Tydfil 005F
44645    Merthyr Tydfil 006G
Name: geography_x, Length: 35672, dtype: object

In [10]:
test_file.head()

,pcd7,pcd8,pcds,dointr,doterm,usertype,oa11cd,lsoa11cd,msoa11cd,ladcd,lsoa11nm,msoa11nm,ladnm,ladnmw
0,AB1 0AA,AB1 0AA,AB1 0AA,198001,199606,0,S00090303,S01006514,S02001237,S12000033,"Cults, Bieldside and Milltimber West - 02","Cults, Bieldside and Milltimber West",Aberdeen City,NaN
1,AB1 0AB,AB1 0AB,AB1 0AB,198001,199606,0,S00090303,S01006514,S02001237,S12000033,"Cults, Bieldside and Milltimber West - 02","Cults, Bieldside and Milltimber West",Aberdeen City,NaN
2,AB1 0AD,AB1 0AD,AB1 0AD,198001,199606,0,S00090399,S01006514,S02001237,S12000033,"Cults, Bieldside and Milltimber West - 02","Cults, Bieldside and Milltimber West",Aberdeen City,NaN
3,AB1 0AE,AB1 0AE,AB1 0AE,199402,199606,0,S00091322,S01006853,S02001296,S12000034,"Dunecht, Durris and Drumoak - 01","Dunecht, Durris and Drumoak",Aberdeenshire,NaN
4,AB1 0AF,AB1 0AF,AB1 0AF,199012,199207,1,S00090299,S01006511,S02001236,S12000033,Culter - 06,Culter,Aberdeen City,NaN


In [11]:
len(census_dfs[0])

35672